In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from PIL import Image
import matplotlib.pyplot as plt

# Helper functions for image loading and preprocessing
def load_image(image_path, size=(28, 28)):
    image = Image.open(image_path).convert('L')
    image = image.resize(size)
    image = np.array(image)
    return image

def plot_images(images, titles, cmap='viridis'):
    num_images = len(images)
    cols = 3
    rows = (num_images + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))
    axes = axes.flatten()
    
    for i in range(num_images):
        axes[i].imshow(images[i], cmap=cmap)
        axes[i].set_title(titles[i])
        axes[i].axis('off')
    
    # Turn off axes for any remaining empty subplots
    for i in range(num_images, len(axes)):
        axes[i].axis('off')
    
    plt.tight_layout()

    plt.show()

# Padding methods
def cyclic_padding(image, pad_width):
    return np.pad(image, pad_width=pad_width, mode='wrap')

def symmetric_padding(image, pad_width):
    return np.pad(image, pad_width=pad_width, mode='symmetric')

# Quantum Convolution (QuC) with padding
def quantum_convolution(image, kernel, padding, method='cyclic'):
    if method == 'cyclic':
        padded_image = cyclic_padding(image, padding)
    elif method == 'symmetric':
        padded_image = symmetric_padding(image, padding)
    
    # Convolution operation
    kernel_size = kernel.shape[0]
    output_size = (padded_image.shape[0] - kernel_size + 1, padded_image.shape[1] - kernel_size + 1)
    output = np.zeros(output_size)
    
    for i in range(output_size[0]):
        for j in range(output_size[1]):
            output[i, j] = np.sum(padded_image[i:i+kernel_size, j:j+kernel_size] * kernel)
    
    return output

# Stride and pooling operations
def conv_with_stride(image, kernel, stride):
    kernel_size = kernel.shape[0]
    output_size = ((image.shape[0] - kernel_size) // stride + 1, (image.shape[1] - kernel_size) // stride + 1)
    output = np.zeros(output_size)
    
    for i in range(0, output_size[0] * stride, stride):
        for j in range(0, output_size[1] * stride, stride):
            if i + kernel_size <= image.shape[0] and j + kernel_size <= image.shape[1]:
                output[i // stride, j // stride] = np.sum(image[i:i+kernel_size, j:j+kernel_size] * kernel)
    
    return output

def conv_with_padding_and_stride(image, kernel, padding, stride):
    padded_image = cyclic_padding(image, padding)
    return conv_with_stride(padded_image, kernel, stride)

# Batch Normalization (BN) operation
def batch_normalization(image, mean, variance, epsilon=1e-5):
    return (image - mean) / np.sqrt(variance + epsilon)

# Weighted kernels and grouped kernels
def apply_weighted_kernels(image, kernels, weights):
    result = np.zeros(image.shape)
    for kernel, weight in zip(kernels, weights):
        result += weight * quantum_convolution(image, kernel, padding=0)
    return result

def apply_grouped_kernels(image, grouped_kernels):
    results = []
    for group in grouped_kernels:
        result = np.zeros(image.shape)
        for kernel in group:
            result += quantum_convolution(image, kernel, padding=0)
        results.append(result)
    return results

# Amplitude Amplification (example placeholder)
def amplitude_amplification(image):
    return image  # Placeholder for actual amplitude amplification implementation

# Entanglement (example placeholder)
def entanglement_operation(image, kernel):
    return quantum_convolution(image, kernel, padding=0)  # Placeholder for actual entanglement implementation

# QFT and its inverse
def quantum_fourier_transform(image):
    # Placeholder for actual QFT implementation
    return np.fft.fft2(image)

def inverse_quantum_fourier_transform(image):
    # Placeholder for actual inverse QFT implementation
    return np.fft.ifft2(image)

# Full QuC with padding and QFT integration
def full_quantum_convolution(image, kernel, padding):
    padded_image = cyclic_padding(image, padding)
    padded_kernel = cyclic_padding(kernel, padding)
    qft_image = quantum_fourier_transform(padded_image)
    qft_kernel = quantum_fourier_transform(padded_kernel)
    qft_result = qft_image * qft_kernel
    result = inverse_quantum_fourier_transform(qft_result)
    return np.real(result)

# Load MNIST dataset and preprocess
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
mnist_image, _ = trainset[0]
q_mnist_image = np.abs(mnist_image.numpy()[0])

# Load and process the Lenna image
lenna_image_path = 'C:/Users/admin/Desktop/lena_256.jpg'
lenna_image = load_image(lenna_image_path)
q_lenna_image = lenna_image / 255.0

# Load and process the Pepper image
pepper_image_path = 'C:/Users/admin/Desktop/pepper.jpg'
pepper_image = load_image(pepper_image_path)
q_pepper_image = pepper_image / 255.0

# Example of using the defined functions
sample_kernel = np.array([[1, 0, -1], [1, 0, -1], [1, 0, -1]])  # Example kernel
padding = 5
stride = 2

# Processing with padding and stride
padded_mnist_image = cyclic_padding(q_mnist_image, pad_width=padding)
conv_result_mnist = quantum_convolution(q_mnist_image, sample_kernel, padding=padding)
conv_stride_result_mnist = conv_with_padding_and_stride(q_mnist_image, sample_kernel, padding=padding, stride=stride)

padded_lenna_image = cyclic_padding(q_lenna_image, pad_width=padding)
conv_result_lenna = quantum_convolution(q_lenna_image, sample_kernel, padding=padding)
conv_stride_result_lenna = conv_with_padding_and_stride(q_lenna_image, sample_kernel, padding=padding, stride=stride)

padded_pepper_image = cyclic_padding(q_pepper_image, pad_width=padding)
conv_result_pepper = quantum_convolution(q_pepper_image, sample_kernel, padding=padding)
conv_stride_result_pepper = conv_with_padding_and_stride(q_pepper_image, sample_kernel, padding=padding, stride=stride)

# Display results in a grid
images = [
    q_mnist_image, conv_result_mnist, conv_stride_result_mnist,
    q_lenna_image, conv_result_lenna, conv_stride_result_lenna,
    q_pepper_image, conv_result_pepper, conv_stride_result_pepper
]
titles = [
    'MNIST Image', 'MNIST Convolution Result', 'MNIST Convolution with Padding and Stride Result',
    'Lenna Image', 'Lenna Convolution Result', 'Lenna Convolution with Padding and Stride Result',
    'Pepper Image', 'Pepper Convolution Result', 'Pepper Convolution with Padding and Stride Result'
]

plot_images(images, titles, cmap='viridis')


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from PIL import Image
import matplotlib.pyplot as plt

# Helper functions for image loading and preprocessing
def load_image(image_path, size=(28, 28)):
    image = Image.open(image_path).convert('L')
    image = image.resize(size)
    image = np.array(image)
    return image

def plot_images(images, titles, cmap='viridis'):
    num_images = len(images)
    cols = 3
    rows = (num_images + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))
    axes = axes.flatten()
    
    for i in range(num_images):
        axes[i].imshow(images[i], cmap=cmap)
        axes[i].set_title(titles[i])
        axes[i].axis('off')
    
    # Turn off axes for any remaining empty subplots
    for i in range(num_images, len(axes)):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

# Padding methods
def cyclic_padding(image, pad_width):
    return np.pad(image, pad_width=pad_width, mode='wrap')

# Quantum Fourier Transform (QFT) and its inverse
def quantum_fourier_transform(image):
    return np.fft.fft2(image)

def inverse_quantum_fourier_transform(image):
    return np.fft.ifft2(image)

# Quantum Convolution with Pre-padding and QFT
def quantum_convolution_with_pre_padding(image, kernel, padding):
    padded_image = cyclic_padding(image, pad_width=padding)
    kernel_size = kernel.shape
    padded_kernel = cyclic_padding(kernel, pad_width=((padded_image.shape[0] - kernel_size[0]) // 2, (padded_image.shape[1] - kernel_size[1]) // 2))
    
    # Ensure the padded kernel and image have the same size
    padded_kernel = np.pad(kernel, pad_width=((0, padded_image.shape[0] - kernel_size[0]), (0, padded_image.shape[1] - kernel_size[1])), mode='constant', constant_values=0)
    
    qft_image = quantum_fourier_transform(padded_image)
    qft_kernel = quantum_fourier_transform(padded_kernel)
    
    qft_result = qft_image * qft_kernel
    result = inverse_quantum_fourier_transform(qft_result)
    
    return np.real(result)

# Load MNIST dataset and preprocess
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
mnist_image, _ = trainset[0]
q_mnist_image = np.abs(mnist_image.numpy()[0])

# Load and process the Lenna image
lenna_image_path = 'C:/Users/admin/Desktop/lena_256.jpg'
lenna_image = load_image(lenna_image_path)
q_lenna_image = lenna_image / 255.0

# Load and process the Pepper image
pepper_image_path = 'C:/Users/admin/Desktop/pepper.jpg'
pepper_image = load_image(pepper_image_path)
q_pepper_image = pepper_image / 255.0

# Example of using the defined functions
sample_kernel = np.array([[1, 0, -1], [1, 0, -1], [1, 0, -1]])  # Example kernel
padding = 10

# Quantum convolution with pre-padding
conv_result_mnist = quantum_convolution_with_pre_padding(q_mnist_image, sample_kernel, padding=padding)
conv_result_lenna = quantum_convolution_with_pre_padding(q_lenna_image, sample_kernel, padding=padding)
conv_result_pepper = quantum_convolution_with_pre_padding(q_pepper_image, sample_kernel, padding=padding)

# Display results in a grid
images = [
    q_mnist_image, conv_result_mnist,
    q_lenna_image, conv_result_lenna,
    q_pepper_image, conv_result_pepper
]
titles = [
    'MNIST Image', 'MNIST Convolution Result',
    'Lenna Image', 'Lenna Convolution Result',
    'Pepper Image', 'Pepper Convolution Result'
]

plot_images(images, titles, cmap='viridis')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import datasets, transforms

# Define the transform for the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
image, _ = trainset[0]
q_image = np.abs(image.numpy()[0])

# Load and process the Lena image
def load_image(path, size=(28, 28)):
    image = Image.open(path).convert('L')
    image = image.resize(size)
    return np.array(image) / 255.0

lenna_image = load_image('C:/Users/admin/Desktop/lena_256.jpg')
download_image = load_image('C:/Users/admin/Desktop/pepper.jpg')

# Define a function to apply the weighted quantum convolution
def quantum_convolution_weighted(q_i, q_k, weights):
    result = np.zeros_like(q_i)
    for k, w in zip(q_k, weights):
        result += w * (k * q_i)
    return result

# Define a function to apply the separable quantum convolution
def quantum_convolution_separable(q_i, q_kx, q_ky):
    result = np.zeros_like(q_i)
    for kx, ky in zip(q_kx, q_ky):
        result += np.outer(kx, ky) * q_i
    return result

# Define a function to apply the dilated quantum convolution
def quantum_convolution_dilated(q_i, q_k, d):
    result = np.zeros_like(q_i)
    kernel_size = q_k[0].shape  # Assuming all kernels have the same size

    for k in q_k:
        dilated_k = np.zeros_like(q_i)
        for i in range(0, q_i.shape[0] - kernel_size[0] + 1, d):
            for j in range(0, q_i.shape[1] - kernel_size[1] + 1, d):
                dilated_k[i:i+kernel_size[0], j:j+kernel_size[1]] = k
        result += dilated_k * q_i
    return result

# Define a function to apply the grouped quantum convolution
def quantum_convolution_grouped(q_i, q_kgroups):
    result = np.zeros_like(q_i)
    for group in q_kgroups:
        group_result = np.zeros_like(q_i)
        for k in group:
            group_result += k * q_i
        result += group_result
    return result

# Example kernels and weights
kernels = [np.random.rand(28, 28) for _ in range(3)]
weights = [0.3, 0.5, 0.2]
kx = [np.random.rand(28) for _ in range(3)]
ky = [np.random.rand(28) for _ in range(3)]
groups = [[np.random.rand(28, 28) for _ in range(2)], [np.random.rand(28, 28) for _ in range(2)]]
dilation_rate = 2

# Apply the quantum convolution operations to the MNIST image
q_weighted = quantum_convolution_weighted(q_image, kernels, weights)
q_separable = quantum_convolution_separable(q_image, kx, ky)
q_dilated = quantum_convolution_dilated(q_image, kernels, dilation_rate)
q_grouped = quantum_convolution_grouped(q_image, groups)

# Plot the results for MNIST image
fig, axes = plt.subplots(1, 5, figsize=(20, 4))
axes[0].imshow(q_image, cmap='viridis')
axes[0].set_title('Original MNIST Image')
axes[1].imshow(q_weighted, cmap='viridis')
axes[1].set_title('Weighted Quantum Convolution')
axes[2].imshow(q_separable, cmap='viridis')
axes[2].set_title('Separable Quantum Convolution')
axes[3].imshow(q_dilated, cmap='viridis')
axes[3].set_title('Dilated Quantum Convolution')
axes[4].imshow(q_grouped, cmap='viridis')
axes[4].set_title('Grouped Quantum Convolution')
plt.show()

# Apply the quantum convolution operations to the Lena image
q_weighted_lenna = quantum_convolution_weighted(lenna_image, kernels, weights)
q_separable_lenna = quantum_convolution_separable(lenna_image, kx, ky)
q_dilated_lenna = quantum_convolution_dilated(lenna_image, kernels, dilation_rate)
q_grouped_lenna = quantum_convolution_grouped(lenna_image, groups)

# Plot the results for Lenna image
fig, axes = plt.subplots(1, 5, figsize=(20, 4))
axes[0].imshow(lenna_image, cmap='viridis')
axes[0].set_title('Original Lenna Image')
axes[1].imshow(q_weighted_lenna, cmap='viridis')
axes[1].set_title('Weighted Quantum Convolution')
axes[2].imshow(q_separable_lenna, cmap='viridis')
axes[2].set_title('Separable Quantum Convolution')
axes[3].imshow(q_dilated_lenna, cmap='viridis')
axes[3].set_title('Dilated Quantum Convolution')
axes[4].imshow(q_grouped_lenna, cmap='viridis')
axes[4].set_title('Grouped Quantum Convolution')
plt.show()

# Apply the quantum convolution operations to the downloaded image
q_weighted_download = quantum_convolution_weighted(download_image, kernels, weights)
q_separable_download = quantum_convolution_separable(download_image, kx, ky)
q_dilated_download = quantum_convolution_dilated(download_image, kernels, dilation_rate)
q_grouped_download = quantum_convolution_grouped(download_image, groups)

# Plot the results for downloaded image
fig, axes = plt.subplots(1, 5, figsize=(20, 4))
axes[0].imshow(download_image, cmap='viridis')
axes[0].set_title('Original Pepper Image')
axes[1].imshow(q_weighted_download, cmap='viridis')
axes[1].set_title('Weighted Quantum Convolution')
axes[2].imshow(q_separable_download, cmap='viridis')
axes[2].set_title('Separable Quantum Convolution')
axes[3].imshow(q_dilated_download, cmap='viridis')
axes[3].set_title('Dilated Quantum Convolution')
axes[4].imshow(q_grouped_download, cmap='viridis')
axes[4].set_title('Grouped Quantum Convolution')

plt.show()


In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from scipy.fftpack import fft2, ifft2

# Function to load and process images
def load_image(path, size=(28, 28)):
    image = Image.open(path).convert('L')
    image = image.resize(size)
    return np.array(image) / 255.0

# Load and preprocess images
# Define the transform for the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
image, _ = trainset[0]
q_image = np.abs(image.numpy()[0])
lenna_image = load_image('C:/Users/admin/Desktop/lena_256.jpg')
pepper_image = load_image('C:/Users/admin/Desktop/pepper.jpg')

# Padding function
def pad_image(image, pad_size):
    padded_image = np.pad(image, pad_size, mode='constant')
    return padded_image

# QFT and inverse QFT functions
def qft(image):
    return fft2(image)

def iqft(image):
    return ifft2(image)

# Convolution in the frequency domain
def quantum_convolution(image, kernel, pad_size):
    padded_image = pad_image(image, pad_size)
    padded_kernel = pad_image(kernel, pad_size)
    
    qft_image = qft(padded_image)
    qft_kernel = qft(padded_kernel)
    
    conv_result = qft_image * qft_kernel
    conv_result_spatial = iqft(conv_result)
    
    return np.abs(conv_result_spatial)

# Example kernel
kernel = np.random.rand(28, 28)

# Apply quantum convolution operations
pad_size = 5

q_conv_mnist = quantum_convolution(mnist_image, kernel, pad_size)
q_conv_lenna = quantum_convolution(lenna_image, kernel, pad_size)
q_conv_pepper = quantum_convolution(pepper_image, kernel, pad_size)

# Plot results
fig, axes = plt.subplots(3, 4, figsize=(20, 16))

# MNIST image results
axes[0, 0].imshow(mnist_image, cmap='viridis')
axes[0, 0].set_title('Original MNIST Image')
axes[0, 1].imshow(pad_image(mnist_image, pad_size), cmap='viridis')
axes[0, 1].set_title('Padded MNIST Image')
axes[0, 2].imshow(np.log1p(np.abs(qft(mnist_image))), cmap='viridis')
axes[0, 2].set_title('QFT of MNIST Image')
axes[0, 3].imshow(q_conv_mnist, cmap='viridis')
axes[0, 3].set_title('Quantum Convolution MNIST')

# Lenna image results
axes[1, 0].imshow(lenna_image, cmap='viridis')
axes[1, 0].set_title('Original Lenna Image')
axes[1, 1].imshow(pad_image(lenna_image, pad_size), cmap='viridis')
axes[1, 1].set_title('Padded Lenna Image')
axes[1, 2].imshow(np.log1p(np.abs(qft(lenna_image))), cmap='viridis')
axes[1, 2].set_title('QFT of Lenna Image')
axes[1, 3].imshow(q_conv_lenna, cmap='viridis')
axes[1, 3].set_title('Quantum Convolution Lenna')

# Pepper image results
axes[2, 0].imshow(pepper_image, cmap='viridis')
axes[2, 0].set_title('Original Pepper Image')
axes[2, 1].imshow(pad_image(pepper_image, pad_size), cmap='viridis')
axes[2, 1].set_title('Padded Pepper Image')
axes[2, 2].imshow(np.log1p(np.abs(qft(pepper_image))), cmap='viridis')
axes[2, 2].set_title('QFT of Pepper Image')
axes[2, 3].imshow(q_conv_pepper, cmap='viridis')
axes[2, 3].set_title('Quantum Convolution Pepper')

plt.tight_layout()

plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft, ifft
from PIL import Image

# Function to load and process images
def load_image(path, size=(28, 28)):
    image = Image.open(path).convert('L')
    image = image.resize(size)
    return np.array(image) / 255.0

# Function to pad the image
def pad_image(image, pad_size):
    return np.pad(image, pad_size, mode='constant')

# Quantum Fourier Transform (QFT)
def qft(x):
    N = len(x)
    return np.fft.fft(x) / np.sqrt(N)

# Inverse Quantum Fourier Transform (IQFT)
def iqft(x):
    N = len(x)
    return np.fft.ifft(x) * np.sqrt(N)

# Applying QFT to the padded image
def apply_qft(image):
    return np.fft.fft2(image) / np.sqrt(image.size)

# Applying inverse QFT to the padded image
def apply_iqft(image):
    return np.fft.ifft2(image) * np.sqrt(image.size)

# Quantum convolution with pre-padding and QFT
def quantum_convolution(image, kernel, pad_size):
    padded_image = pad_image(image, pad_size)
    padded_kernel = pad_image(kernel, pad_size)
    
    qft_image = apply_qft(padded_image)
    iqft_kernel = apply_iqft(padded_kernel)
    
    convolved = qft_image * iqft_kernel
    result = apply_iqft(convolved)
    
    return np.abs(result)

# Load the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
image, _ = trainset[0]
q_image = np.abs(image.numpy()[0])

# Load and process an image
def load_image(path, size=(28, 28)):
    image = Image.open(path).convert('L')
    image = image.resize(size)
    return np.array(image) / 255.0

lenna_image = load_image('C:/Users/admin/Desktop/lena_256.jpg')
download_image = load_image('C:/Users/admin/Desktop/pepper.jpg')
# Define padding size and kernel
pad_size = ((2, 2), (2, 2))
kernel = np.random.rand(28, 28)

# Apply quantum convolution
q_conv_mnist = quantum_convolution(mnist_image, kernel, pad_size)
q_conv_lenna = quantum_convolution(lenna_image, kernel, pad_size)
q_conv_download = quantum_convolution(download_image, kernel, pad_size)

# Plot the results
fig, axes = plt.subplots(3, 3, figsize=(15, 15))

axes[0, 0].imshow(mnist_image, cmap='viridis')
axes[0, 0].set_title('Original MNIST Image')
axes[0, 1].imshow(np.abs(qft(mnist_image)), cmap='viridis')
axes[0, 1].set_title('QFT of MNIST Image')
axes[0, 2].imshow(q_conv_mnist, cmap='viridis')
axes[0, 2].set_title('Quantum Convolution MNIST')

axes[1, 0].imshow(lenna_image, cmap='viridis')
axes[1, 0].set_title('Original Lenna Image')
axes[1, 1].imshow(np.abs(qft(lenna_image)), cmap='viridis')
axes[1, 1].set_title('QFT of Lenna Image')
axes[1, 2].imshow(q_conv_lenna, cmap='viridis')
axes[1, 2].set_title('Quantum Convolution Lenna')

axes[2, 0].imshow(download_image, cmap='viridis')
axes[2, 0].set_title('Original Downloaded Image')
axes[2, 1].imshow(np.abs(qft(download_image)), cmap='viridis')
axes[2, 1].set_title('QFT of Downloaded Image')
axes[2, 2].imshow(q_conv_download, cmap='viridis')
axes[2, 2].set_title('Quantum Convolution Downloaded Image')

plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from scipy.fftpack import fft2, ifft2

# Load and process images
def load_image(path, size=(28, 28)):
    image = Image.open(path).convert('L')
    image = image.resize(size)
    return np.array(image) / 255.0

mnist_image = np.abs(datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transforms.Compose([transforms.ToTensor()]))[0][0].numpy()[0])
lenna_image = load_image('C:/Users/admin/Desktop/lena_256.jpg')
pepper_image = load_image('C:/Users/admin/Desktop/pepper.jpg')

# Padding function
def pad_image(image, pad_size):
    padded_image = np.pad(image, ((0, pad_size[0] - image.shape[0]), (0, pad_size[1] - image.shape[1])), mode='constant', constant_values=0)
    return padded_image

def pad_kernel(kernel, pad_size):
    padded_kernel = np.pad(kernel, ((0, pad_size[0] - kernel.shape[0]), (0, pad_size[1] - kernel.shape[1])), mode='constant', constant_values=0)
    return padded_kernel

# Define Quantum Fourier Transform (QFT) and its inverse
def qft_2d(matrix):
    return fft2(matrix)

def qft_inv_2d(matrix):
    return ifft2(matrix)

# Quantum Convolution
def quantum_convolution(image, kernel, pad_size):
    padded_image = pad_image(image, pad_size)
    padded_kernel = pad_kernel(kernel, pad_size)
    
    # QFT
    qft_image = qft_2d(padded_image)
    qft_kernel = qft_2d(padded_kernel)
    
    # Convolution in frequency domain
    qft_convolution = qft_image * qft_kernel
    
    # Inverse QFT
    result = qft_inv_2d(qft_convolution)
    
    return np.abs(result)

# Example kernels
kernel_size = (28, 28)
kernels = [np.random.rand(*kernel_size) for _ in range(3)]
pad_size = (64, 64)  # Example padding size

# Apply quantum convolution to MNIST image
results_mnist = [quantum_convolution(mnist_image, k, pad_size) for k in kernels]

# Apply quantum convolution to Lena image
results_lenna = [quantum_convolution(lenna_image, k, pad_size) for k in kernels]

# Apply quantum convolution to Pepper image
results_pepper = [quantum_convolution(pepper_image, k, pad_size) for k in kernels]

# Plot the results
def plot_results(image, results, title):
    fig, axes = plt.subplots(1, len(results) + 1, figsize=(20, 4))
    axes[0].imshow(image, cmap='gray')
    axes[0].set_title(f'Original {title}')
    for i, result in enumerate(results):
        axes[i + 1].imshow(result, cmap='gray')
        axes[i + 1].set_title(f'Quantum Conv {i+1}')
    plt.show()

plot_results(mnist_image, results_mnist, 'MNIST Image')
plot_results(lenna_image, results_lenna, 'Lenna Image')
plot_results(pepper_image, results_pepper, 'Pepper Image')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from scipy.fftpack import fft2, ifft2
from torchvision import datasets, transforms

# Load and process images
def load_image(path, size=(28, 28)):
    image = Image.open(path).convert('L')
    image = image.resize(size)
    return np.array(image) / 255.0

# Load MNIST image
transform = transforms.Compose([transforms.ToTensor()])
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
mnist_image = np.abs(trainset[0][0].numpy()[0])

# Load Lena and Pepper images
lenna_image = load_image('C:/Users/admin/Desktop/lena_256.jpg')
pepper_image = load_image('C:/Users/admin/Desktop/pepper.jpg')

# Padding function
def pad_image(image, pad_size):
    padded_image = np.pad(image, ((0, pad_size[0] - image.shape[0]), (0, pad_size[1] - image.shape[1])), mode='constant', constant_values=0)
    return padded_image

def pad_kernel(kernel, pad_size):
    padded_kernel = np.pad(kernel, ((0, pad_size[0] - kernel.shape[0]), (0, pad_size[1] - kernel.shape[1])), mode='constant', constant_values=0)
    return padded_kernel

# Define Quantum Fourier Transform (QFT) and its inverse
def qft_2d(matrix):
    return fft2(matrix)

def qft_inv_2d(matrix):
    return ifft2(matrix)

# Quantum Convolution
def quantum_convolution(image, kernel, pad_size):
    padded_image = pad_image(image, pad_size)
    padded_kernel = pad_kernel(kernel, pad_size)
    
    # QFT
    qft_image = qft_2d(padded_image)
    qft_kernel = qft_2d(padded_kernel)
    
    # Convolution in frequency domain
    qft_convolution = qft_image * qft_kernel
    
    # Inverse QFT
    result = qft_inv_2d(qft_convolution)
    
    return np.abs(result)

# Define different kernels for Conv1, Conv2, Conv3
kernel_size = (28, 28)
pad_size = (64, 64)  # Example padding size

# Conv1: Kernel with high frequency components
kernel_conv1 = np.sin(np.linspace(0, 10 * np.pi, kernel_size[0] * kernel_size[1]).reshape(kernel_size))

# Conv2: Kernel with low frequency components
kernel_conv2 = np.exp(-np.linspace(0, 5, kernel_size[0] * kernel_size[1]).reshape(kernel_size))

# Conv3: Random kernel with a mix of frequencies
kernel_conv3 = np.random.rand(*kernel_size)

kernels = [kernel_conv1, kernel_conv2, kernel_conv3]

# Apply quantum convolution to MNIST image
results_mnist = [quantum_convolution(mnist_image, k, pad_size) for k in kernels]

# Apply quantum convolution to Lena image
results_lenna = [quantum_convolution(lenna_image, k, pad_size) for k in kernels]

# Apply quantum convolution to Pepper image
results_pepper = [quantum_convolution(pepper_image, k, pad_size) for k in kernels]

# Plot the results
def plot_results(image, results, title):
    fig, axes = plt.subplots(1, len(results) + 1, figsize=(20, 4))
    axes[0].imshow(image, cmap='gray')
    axes[0].set_title(f'Original {title}')
    for i, result in enumerate(results):
        axes[i + 1].imshow(result, cmap='gray')
        axes[i + 1].set_title(f'Quantum Conv {i+1}')
    plt.show()

plot_results(mnist_image, results_mnist, 'MNIST Image')
plot_results(lenna_image, results_lenna, 'Lenna Image')
plot_results(pepper_image, results_pepper, 'Pepper Image')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from scipy.fftpack import fft2, ifft2
from torchvision import datasets, transforms

# Load and process images
def load_image(path, size=(28, 28)):
    image = Image.open(path).convert('L')
    image = image.resize(size)
    return np.array(image) / 255.0

# Load MNIST image
transform = transforms.Compose([transforms.ToTensor()])
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
mnist_image = np.abs(trainset[0][0].numpy()[0])

# Load Lena and Pepper images
lenna_image = load_image('C:/Users/admin/Desktop/lena_256.jpg')
pepper_image = load_image('C:/Users/admin/Desktop/pepper.jpg')

# Padding function
def pad_image(image, pad_size):
    padded_image = np.pad(image, ((0, pad_size[0] - image.shape[0]), (0, pad_size[1] - image.shape[1])), mode='constant', constant_values=0)
    return padded_image

def pad_kernel(kernel, pad_size):
    padded_kernel = np.pad(kernel, ((0, pad_size[0] - kernel.shape[0]), (0, pad_size[1] - kernel.shape[1])), mode='constant', constant_values=0)
    return padded_kernel

# Define Quantum Fourier Transform (QFT) and its inverse
def qft_2d(matrix):
    return fft2(matrix)

def qft_inv_2d(matrix):
    return ifft2(matrix)

# Quantum Convolution
def quantum_convolution(image, kernel, pad_size):
    padded_image = pad_image(image, pad_size)
    padded_kernel = pad_kernel(kernel, pad_size)
    
    # QFT
    qft_image = qft_2d(padded_image)
    qft_kernel = qft_2d(padded_kernel)
    
    # Convolution in frequency domain
    qft_convolution = qft_image * qft_kernel
    
    # Inverse QFT
    result = qft_inv_2d(qft_convolution)
    
    return np.abs(result)

# Define different kernels for Conv1, Conv2, Conv3
def create_kernels(size):
    # Conv1: High-frequency kernel
    x, y = np.meshgrid(np.linspace(0, 1, size[0]), np.linspace(0, 1, size[1]))
    k_conv1 = np.sin(2 * np.pi * np.sqrt(x**2 + y**2) * 10)
    
    # Conv2: Low-frequency kernel
    x, y = np.meshgrid(np.linspace(-1, 1, size[0]), np.linspace(-1, 1, size[1]))
    k_conv2 = np.exp(-(x**2 + y**2) / (2 * 0.5**2))
    
    # Conv3: Random kernel
    k_conv3 = np.random.rand(*size)
    
    return [k_conv1, k_conv2, k_conv3]

kernel_size = (64, 64)  # Padding size
kernels = create_kernels(kernel_size)

# Apply quantum convolution to MNIST image
results_mnist = [quantum_convolution(mnist_image, k, kernel_size) for k in kernels]

# Apply quantum convolution to Lena image
results_lenna = [quantum_convolution(lenna_image, k, kernel_size) for k in kernels]

# Apply quantum convolution to Pepper image
results_pepper = [quantum_convolution(pepper_image, k, kernel_size) for k in kernels]

# Plot the results
def plot_results(image, results, title):
    fig, axes = plt.subplots(1, len(results) + 1, figsize=(20, 16))
    axes[0].imshow(image, cmap='viridis')
    axes[0].set_title(f'Original {title}')
    for i, result in enumerate(results):
        axes[i + 1].imshow(result, cmap='viridis')
        axes[i + 1].set_title(f'Quantum Conv {i+1}')

    
    plt.tight_layout()
    plt.show()

plot_results(mnist_image, results_mnist, 'MNIST Image')
plot_results(lenna_image, results_lenna, 'Lenna Image')
plot_results(pepper_image, results_pepper, 'Pepper Image')
    




In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from scipy.fftpack import fft2, ifft2

# Load and process images
def load_image(path, size=(28, 28)):
    image = Image.open(path).convert('L')
    image = image.resize(size)
    return np.array(image) / 255.0

def pad_image(image, pad_size):
    return np.pad(image, ((0, pad_size[0] - image.shape[0]), (0, pad_size[1] - image.shape[1])), mode='constant')

def pad_kernel(kernel, pad_size):
    return np.pad(kernel, ((0, pad_size[0] - kernel.shape[0]), (0, pad_size[1] - kernel.shape[1])), mode='constant')

def qft_2d(matrix):
    return fft2(matrix)

def qft_inv_2d(matrix):
    return ifft2(matrix)

# Define convolution kernels
def kernel_conv1(size):
    x, y = np.indices(size)
    return np.sin(2 * np.pi * x / size[0]) * np.sin(2 * np.pi * y / size[1])

def kernel_conv2(size):
    x, y = np.indices(size)
    center = (size[0] // 2, size[1] // 2)
    sigma = 10
    return np.exp(-np.sqrt((x - center[0])**2 + (y - center[1])**2) / sigma)

def kernel_conv3(size):
    return np.random.rand(size[0], size[1])

# Convolution operation using QFT
def quantum_convolution(image, kernel, pad_size):
    padded_image = pad_image(image, pad_size)
    padded_kernel = pad_kernel(kernel, pad_size)
    
    qft_image = qft_2d(padded_image)
    qft_kernel = qft_2d(padded_kernel)
    
    qft_convolution = qft_image * qft_kernel
    
    result = qft_inv_2d(qft_convolution)
    
    return np.abs(result)

# Define parameters
kernel_size = (28, 28)
pad_size = (64, 64)  # Example padding size

# Create kernels
kernel1 = kernel_conv1(kernel_size)
kernel2 = kernel_conv2(kernel_size)
kernel3 = kernel_conv3(kernel_size)

kernels = [kernel1, kernel2, kernel3]

# Load MNIST image
transform = transforms.Compose([transforms.ToTensor()])
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
mnist_image = np.abs(trainset[0][0].numpy()[0])

# Load Lena and Pepper images
lenna_image = load_image('C:/Users/admin/Desktop/lena_256.jpg')
pepper_image = load_image('C:/Users/admin/Desktop/pepper.jpg')

# Apply quantum convolution to images
results_mnist = [quantum_convolution(mnist_image, k, pad_size) for k in kernels]
results_lenna = [quantum_convolution(lenna_image, k, pad_size) for k in kernels]
results_pepper = [quantum_convolution(pepper_image, k, pad_size) for k in kernels]

# Plot results
def plot_results(image, results, title):
    fig, axes = plt.subplots(1, len(results) + 1, figsize=(20, 4))
    axes[0].imshow(image, cmap='viridis')
    axes[0].set_title(f'Original {title}')
    for i, result in enumerate(results):
        axes[i + 1].imshow(result, cmap='viridis')
        axes[i + 1].set_title(f'Quantum Conv {i+1}')
    plt.show()

plot_results(mnist_image, results_mnist, 'MNIST Image')
plot_results(lenna_image, results_lenna, 'Lenna Image')
plot_results(pepper_image, results_pepper, 'Pepper Image')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft2, ifft2
from torchvision import datasets, transforms
from PIL import Image

# Function to load and convert images to grayscale
def load_image(path):
    image = Image.open(path).convert('L')
    return np.array(image)

# Pad function
def pad_image(image, pad_size):
    padded_image = np.zeros(pad_size)
    padded_image[:image.shape[0], :image.shape[1]] = image
    return padded_image

def pad_kernel(kernel, pad_size):
    padded_kernel = np.zeros(pad_size)
    padded_kernel[:kernel.shape[0], :kernel.shape[1]] = kernel
    return padded_kernel

# Quantum Fourier Transform (QFT)
def qft_2d(image):
    return fft2(image)

# Inverse Quantum Fourier Transform (QFT†)
def iqft_2d(image):
    return ifft2(image)

# Quantum convolution operation
def quantum_convolution(image, kernel):
    pad_size = (max(image.shape[0], kernel.shape[0]), max(image.shape[1], kernel.shape[1]))
    padded_image = pad_image(image, pad_size)
    padded_kernel = pad_kernel(kernel, pad_size)

    qft_image = qft_2d(padded_image)
    iqft_kernel = iqft_2d(padded_kernel)

    conv_result = qft_image * iqft_kernel
    result_image = np.real(ifft2(conv_result))

    return result_image

# Example kernels
kernels = [
    np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]]),  # Sharpen
    np.array([[1, 1, 1], [1, -7, 1], [1, 1, 1]]),  # Edge enhancement
    np.array([[1, 2, 1], [2, 4, 2], [1, 2, 1]]) / 16  # Gaussian blur
]

# Load MNIST image
transform = transforms.Compose([transforms.ToTensor()])
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
mnist_image = np.abs(trainset[0][0].numpy()[0])

# Load Lena and Pepper images
lenna_image = load_image('C:/Users/admin/Desktop/lena_256.jpg')
pepper_image = load_image('C:/Users/admin/Desktop/pepper.jpg')

# Apply quantum convolution to images
results_mnist = [quantum_convolution(mnist_image, k) for k in kernels]
results_lenna = [quantum_convolution(lenna_image, k) for k in kernels]
results_pepper = [quantum_convolution(pepper_image, k) for k in kernels]

# Display results
fig, axs = plt.subplots(3, 4, figsize=(15, 12))

# MNIST
axs[0, 0].imshow(mnist_image, cmap='gray')
axs[0, 0].set_title('Original MNIST')
for i in range(3):
    axs[0, i+1].imshow(results_mnist[i], cmap='gray')
    axs[0, i+1].set_title(f'Filtered MNIST {i+1}')

# Lenna
axs[1, 0].imshow(lenna_image, cmap='gray')
axs[1, 0].set_title('Original Lenna')
for i in range(3):
    axs[1, i+1].imshow(results_lenna[i], cmap='gray')
    axs[1, i+1].set_title(f'Filtered Lenna {i+1}')

# Pepper
axs[2, 0].imshow(pepper_image, cmap='gray')
axs[2, 0].set_title('Original Pepper')
for i in range(3):
    axs[2, i+1].imshow(results_pepper[i], cmap='gray')
    axs[2, i+1].set_title(f'Filtered Pepper {i+1}')

plt.tight_layout()

plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft2, ifft2
from torchvision import datasets, transforms
from PIL import Image

# Define controlled rotation gates
def crx(theta):
    return np.array([
        [1, 0, 0, 0],
        [0, np.cos(theta / 2), 0, -1j * np.sin(theta / 2)],
        [0, 0, 1, 0],
        [0, -1j * np.sin(theta / 2), 0, np.cos(theta / 2)]
    ])

def cry(theta):
    return np.array([
        [1, 0, 0, 0],
        [0, np.cos(theta / 2), 0, -np.sin(theta / 2)],
        [0, 0, 1, 0],
        [0, -np.sin(theta / 2), 0, np.cos(theta / 2)]
    ])

def crz(theta):
    return np.array([
        [1, 0, 0, 0],
        [0, np.exp(-1j * theta / 2), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, np.exp(1j * theta / 2)]
    ])

# Apply controlled rotation gates to each pair of qubits
def apply_crx(theta, state):
    num_qubits = int(np.log2(len(state)))
    gate = crx(theta)
    for i in range(0, len(state), 4):
        state[i:i+4] = np.dot(gate, state[i:i+4])
    return state

def apply_cry(theta, state):
    num_qubits = int(np.log2(len(state)))
    gate = cry(theta)
    for i in range(0, len(state), 4):
        state[i:i+4] = np.dot(gate, state[i:i+4])
    return state

def apply_crz(theta, state):
    num_qubits = int(np.log2(len(state)))
    gate = crz(theta)
    for i in range(0, len(state), 4):
        state[i:i+4] = np.dot(gate, state[i:i+4])
    return state

# Load and convert images
def load_image(path):
    image = Image.open(path).convert('L')
    return np.array(image)

# Pad function
def pad_to_size(image, size):
    padded_image = np.zeros(size)
    padded_image[:image.shape[0], :image.shape[1]] = image
    return padded_image

# Quantum Fourier Transform (QFT) and its Inverse
def qft_2d(image):
    return fft2(image)

def iqft_2d(image):
    return ifft2(image)

# Quantum convolution operation
def quantum_convolution(image, kernel):
    pad_size = (max(image.shape[0], kernel.shape[0]), max(image.shape[1], kernel.shape[1]))
    padded_image = pad_to_size(image, pad_size)
    padded_kernel = pad_to_size(kernel, pad_size)

    qft_image = qft_2d(padded_image)
    iqft_kernel = iqft_2d(padded_kernel)

    conv_result = qft_image * iqft_kernel
    result_image = np.real(ifft2(conv_result))

    return result_image

# Quantum image state preparation
def prepare_state(image):
    flattened = image.flatten()
    norm = np.linalg.norm(flattened)
    state = flattened / norm
    return state

# Ensure image and kernel dimensions match
def match_dimensions(image, kernel):
    if image.shape != kernel.shape:
        size = (max(image.shape[0], kernel.shape[0]), max(image.shape[1], kernel.shape[1]))
        image = pad_to_size(image, size)
        kernel = pad_to_size(kernel, size)
    return image, kernel

# Resize image to the nearest power of 2
def resize_to_power_of_2(image):
    def next_power_of_2(x):
        return 1 if x == 0 else 2**(x - 1).bit_length()

    new_size = (next_power_of_2(image.shape[0]), next_power_of_2(image.shape[1]))
    return pad_to_size(image, new_size)

# Convolution with controlled rotation gates
def quantum_convolution_with_rotation(image, kernel, theta_x, theta_y, theta_z):
    image = resize_to_power_of_2(image)
    image, kernel = match_dimensions(image, kernel)

    qft_image = qft_2d(image)
    iqft_kernel = iqft_2d(kernel)

    conv_result = qft_image * iqft_kernel
    result_image = np.real(ifft2(conv_result))

    # Prepare quantum state from image
    state = prepare_state(result_image)

    # Ensure the state has a length that is a power of 2 for the rotation gates
    num_qubits = int(np.log2(len(state)))
    state = np.reshape(state, (2**num_qubits, 1))

    # Apply controlled rotation gates iteratively
    state = apply_crx(theta_x, state)
    state = apply_cry(theta_y, state)
    state = apply_crz(theta_z, state)

    # Return to spatial domain
    final_result = np.reshape(state, result_image.shape)
    return final_result

# Example kernels
kernels = [
    np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]]),  # Sharpen
    np.array([[1, 1, 1], [1, -7, 1], [1, 1, 1]]),  # Edge enhancement
    np.array([[1, 2, 1], [2, 4, 2], [1, 2, 1]]) / 16  # Gaussian blur
]

# Load MNIST image
transform = transforms.Compose([transforms.ToTensor()])
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
mnist_image = np.abs(trainset[0][0].numpy()[0])

# Load Lenna and Pepper images
lenna_image = load_image('C:/Users/admin/Desktop/lena_256.jpg')
pepper_image = load_image('C:/Users/admin/Desktop/pepper.jpg')

# Apply quantum convolution with rotation gates to images
theta_x, theta_y, theta_z = np.pi/4, np.pi/6, np.pi/3  # Example rotation angles
results_mnist = [quantum_convolution_with_rotation(mnist_image, k, theta_x, theta_y, theta_z) for k in kernels]
results_lenna = [quantum_convolution_with_rotation(lenna_image, k, theta_x, theta_y, theta_z) for k in kernels]
results_pepper = [quantum_convolution_with_rotation(pepper_image, k, theta_x, theta_y, theta_z) for k in kernels]

# Display results
fig, axs = plt.subplots(3, 4, figsize=(15, 12))

# MNIST
axs[0, 0].imshow(mnist_image, cmap='gray')
axs[0, 0].set_title('Original MNIST')
for i in range(3):
    axs[0, i+1].imshow(results_mnist[i], cmap='gray')
    axs[0, i+1].set_title(f'Filtered MNIST {i+1}')

# Lenna
axs[1, 0].imshow(lenna_image, cmap='gray')
axs[1, 0].set_title('Original Lenna')
for i in range(3):
    axs[1, i+1].imshow(results_lenna[i], cmap='gray')
    axs[1, i+1].set_title(f'Filtered Lenna {i+1}')

# Pepper
axs[2, 0].imshow(pepper_image, cmap='gray')
axs[2, 0].set_title('Original Pepper')
for i in range(3):
    axs[2, i+1].imshow(results_pepper[i], cmap='gray')
    axs[2, i+1].set_title(f'Filtered Pepper {i+1}')

plt.tight_layout()
plt.show()
